In [ ]:
import dash_leaflet as dl
from dash import Dash, html, dcc, Input, Output, dash_table

from mongo_crud import MongoCRUD

import pandas as pd
import numpy as np

import plotly.express as px

###########################
# Data Manipulation / Model
###########################

# define the database query that each rescue type will use
def searchTemplate(rescueType):
    
    return { 
                'breed' : {'$in': rescueType.breeds},
             
                'sex_upon_outcome' : rescueType.sex,
        
                'age_upon_outcome_in_weeks' : {'$gte': rescueType.minAge},
                'age_upon_outcome_in_weeks' : {'$lte': rescueType.maxAge},
            }

# define objects for each rescue type
class WaterRescue():
    
    breeds = [
                'Labrador Retriever Mix',
                'Chesapeake Bay Retriever',
                'Newfoundland',
            ]
    
    sex = 'Intact Female'
    
    minAge = 26
    maxAge = 156
    
class MountainRescue():
    
    breeds = [
                'German Shepherd',
                'Alaskan Malamute',
                'Old English Sheepdog',
                'Siberian Husky',
                'Rottweiler',
            ]
    
    sex = 'Intact Male'
    
    minAge = 26
    maxAge = 156
    
class DisasterRescue():
    
    breeds = [
                'Doberman Pinscher',
                'German Shepherd',
                'Golden Retriever',
                'Bloodhound',
                'Rottweiler',
            ]
    
    sex = 'Intact Male'
    
    minAge = 20
    maxAge = 300

# return a new pandas dataframe by running a database search
def searchDatabase(searchTerm):
    data = pd.DataFrame.from_records(shelter.read(searchTerm))
    data = data.drop('_id', axis=1) # drop _id column, only useful for mongoDB and doesnt seem to work with df.to_dict
    return data


# create connection to animals database
username = "aacuser"
password = "aacuserpassword"
shelter = MongoCRUD('AAC.animals', username, password)

# populate dataframe at start with all
df = searchDatabase({})

# create a custom ordered list of columns contained in the dataframe
columns_ordered = ['animal_id', 'animal_type', 'breed', 'name', 'outcome_type', 'outcome_subtype', 'age_upon_outcome']
rest_of_columns = [col for col in df.columns if col not in columns_ordered]
display_columns = columns_ordered + rest_of_columns

#########################
# Dashboard Layout / View
#########################

# new dash app
app = Dash('Project Two')

app.layout = html.Div([
    
    # HEADER
    html.Div([ # top div (three child divs: left, right, and middle)
    
    # LOGO
        html.Div(html.Img(src='assets\Grazioso_Salvare_Logo.png',
             height='100px', width='100px',), 
             style={'float':'right', 'width': '25%', 'height': '100px', 'text-align': 'right'}),
        
        html.Div(style={'float':'left', 'width': '25%', 'height': '100px'}),
        
    # TITLE
        html.Div(html.B(html.H1('SNHU CS-340 Dashboard')),
             style={'text-align':'center'}),
    ], 
    style={'height': '105px', 'width': '100%'},
    ),
    
    # ANIMAL MAP
    html.Div([ # middle div
        
        html.Div([ # left div of middle
            dl.Map( [dl.TileLayer(), dl.LayerGroup(id="layer")],
                id="map", 
                style={'height': '500px'},
                center=[30.263,-97.744], # austin tx
                zoom=9
              ),
    
    # PIE CHART
        dcc.Graph(id="pie_chart")
        ],
        
        style={'width': '39%', 'height': '805px', 'float':'left', 'text-align' : 'left'},
        ),
    
    # FILTER
        html.Div([ # right div of middle
            html.Div([
            dcc.RadioItems(['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'], 'Reset', id="filter"),
            html.Hr(),
            ]),
        
    # DATA TABLE
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in display_columns
                ],
                data=df.to_dict('records'),
                style_cell={'textAlign': 'right'},
                sort_action='native',
                filter_action='native',
                page_action='native',
                page_current=0,
                page_size=20,
                style_table={'overflowX':'scroll'},
                ),
            ],
        
        style={'float':'right', 'margin': 'auto', 'max-width': '59%', 'min-height': '805px'}),
        ]),
    
    # SIGNATURE
        html.Div( # lower div
            html.P("SNHU CS340 by Craig O'Loughlin (08/14/2022)"),
            style={'text-align': 'right'}),
    ])
    

#########################
# Controls (Callbacks)
#########################

# update the map pinpoint based on first animal on table page view or selected row
@app.callback(
    Output("layer", "children"), 
    Input('datatable-id', 'derived_viewport_data'),
    Input('datatable-id', 'active_cell'))
def update_map(derived_viewport_data, active_cell):
    
    # get the active cell row or default 0
    row = 0
    if active_cell is not None:
        row = active_cell['row']
    
    # get the current dataview
    dff = pd.DataFrame.from_dict(derived_viewport_data)
    
    # get latitude and longitude of first animal in data view (row 0, columns 13 and 14)
    lat = float(dff.iloc[row,13])
    long = float(dff.iloc[row,14])
    
    # get animal name and breed
    animal_name = '(no name in database)' if dff.iloc[row,9] == "" else dff.iloc[0,9]
    animal_breed = '(no breed in database)' if dff.iloc[row,4] == "" else dff.iloc[0,4]
    
    # return a new marker at animal position
    return [
            dl.Marker(position=[lat,long], children=[
                dl.Tooltip(animal_breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name),
                ])
            ])
           ]

# highlight the active cell's row
@app.callback(Output('datatable-id', 'style_data_conditional'),
              Input('datatable-id', 'active_cell'))
def highlight_row(active_cell):
    
    return [
        { # this sets the entire row blue
            'if': {
                'row_index' : active_cell['row'],
            },
            'backgroundColor': 'dodgerblue'
        },
        { # this turns the active cell indicator blue and removes border
            'if': {
                'state' : 'selected',
            },
            'backgroundColor': 'dodgerblue',
            'border' : '0px solid blue'
        },
    ] if active_cell is not None else []

# update the datatable when filter options change
@app.callback(Output('datatable-id', 'data'), Input('filter', 'value'))
def update_table(value):
    
    if (value == 'Water Rescue'):
        df = searchDatabase(searchTemplate(WaterRescue()))
        
    if (value == 'Mountain Rescue'):
        df = searchDatabase(searchTemplate(MountainRescue()))
            
    if (value == 'Disaster Rescue'):
        df = searchDatabase(searchTemplate(DisasterRescue()))
        
    if (value == 'Reset'):
        df = searchDatabase({})
        
    return df.to_dict('records')

# Update pie chart based on datatable
@app.callback(Output("pie_chart", "figure"), Input("datatable-id", "data"))
def generate_pie_chart(data):
    
    fig = px.pie(data, names='breed', height=300)
    
    # hide excessive markers if too many breeds
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return fig

# RUN
if __name__ == '__main__':
    app.run_server()